## Trinity assembly

In [ ]:
%run ../config/init.py

### Loading data from {{ cookiecutter.dataset_name }}/sample_table.csv accession list

The file `{{ cookiecutter.dataset_name }}/sample_table.cs` should contains a single column with all SRA IDs to be processed.

In [ ]:
data_dir = os.path.join(DATA, DATASET)
result_dir = working_dir(os.path.join(RESULTS, DATASET, 'trinity_assembly'))

sra_df = pandas.read_csv(os.path.join(DATA, DATASET, 'sample_table.csv'), header=None)
sra_df

## Testing gcloud configuration

### Requirements

#### [Cloud SDK](https://cloud.google.com/sdk)


Run *gcloud init* to initialize the gcloud environment and follow its instructions:

 `$ gcloud init`

In [ ]:
account = !gcloud config get-value account
account = ''.join(account)
project = !gcloud config get-value project
project = ''.join(project)
if account != '(unset)' and project != '(unset)':
    print('Using account: {} with project: {}'.format(account, project))
else:
    print('Please, configure Cloud SDK before running this notebook')
    print('Open a Terminal and run: gcloud init')

### Defining variables

Edit GCP zone and region variable accordingly to your geographical location.

In [ ]:
REGION = 'us-east4'
ZONE = 'us-east4-c'

### Retrieve GCP storage bucket

In [ ]:
bucket_list = !gsutil ls
bucket = None

prefix = 'gs://{}-vector-'.format(DATASET.lower())
for l in bucket_list:
    if prefix in l:
        bucket = l.replace('gs://{}-vector-'.format(DATASET.lower()),'').replace('/','')
        break

vector_bucket  = '{}-vector-{}'.format(DATASET.lower(),bucket)
print('vector bucket: {0}'.format(vector_bucket))

out_bucket = '{}-trinity-{}'.format(DATASET.lower(), bucket)
bucket_list = !gsutil ls gs://{out_bucket}
if ''.join(bucket_list).startswith('BucketNotFoundException'):
    !gsutil mb gs://{out_bucket}

## Submitting job

In [ ]:
operations = {}
PIPELINE = os.path.join(BIN, 'gcp', 'pipeline-trinity.json')

os.chdir(result_dir)
       
if os.path.exists('operations-trinity.tsv'):
    operations['logs'] = {}
    with open('operations-trinity.tsv') as fin:
        operations['operations'] = fin.readline().strip()
else:
    trinity_yml = {
        "max_memory" : "500G",
        "CPU": 128,
        "output": "trinity",
        "seqType": "fq",
{% if cookiecutter.sequencing_technology == 'paired-end' %}
        "left": [],
        "right": []
{% else %}
        "single": []
{% endif %}
    }
    for s in sra_df[0].unique():
{% if cookiecutter.sequencing_technology == 'paired-end' %}
        trinity_yml['left'].append(
            {'class': 'File', 'path': '/data/{}_clean_noCont_1.fastq.gz'.format(s)}
        )
        trinity_yml['right'].append(
            {'class': 'File', 'path': '/data/{}_clean_noCont_2.fastq.gz'.format(s)}
        )
{% else %}
        trinity_yml['single'].append(
            {'class': 'File', 'path': '/data/{}_clean_noCont.fastq.gz'.format(s)}
        )
{% endif %}
    write_to_yaml(trinity_yml, "trinity.yml")  
    !gsutil cp trinity.yml gs://{vector_bucket}
    a = !gcloud beta lifesciences pipelines run --pipeline-file={PIPELINE} --env-vars=OUTBUCKET={out_bucket},INBUCKET={vector_bucket}
    if len(a) == 1 and a[0].startswith('Running'):
        a = a[0].replace('].','').split('/')[5]
        operations['operations'] = a
        with open('operations-trinity.tsv', 'w') as fout:
            fout.write('{}\n'.format(a))
    else:
        print("ERROR:\n" + a)
print('Operation: ' +  operations['operations'])        

In [ ]:
id = operations['operations']
a = !gcloud beta lifesciences operations describe --format=json {id}
l = json.loads(''.join(a))
if 'done' in l and l['done'] and 'error' not in l:
    operations['logs'][r['sample']] = l
    with gzip.GzipFile('{}_trinity.json.gz'.format(r['sample']), 'w') as fout:   # 4. gzip
        fout.write(json.dumps(l, indent=2).encode('utf-8'))
    ts = get_gpc_starttimestamp(l)
    ts = datetime.strptime(ts.split('.')[0], "%Y-%m-%dT%H:%M:%S")
    te = datetime.strptime(l['metadata']['endTime'].split('.')[0], "%Y-%m-%dT%H:%M:%S")
    elapsed = (te - ts)/pandas.Timedelta('1 minute')
    MACHINE_PRICE = 1.308544 # n2d-standard-128 1.308544 Preemptible
    print('Computig cost: $ {:.2f}'.format(elapsed * MACHINE_PRICE/60))
else:
    print("ERROR")
    print(l['error'])

## Downloading assembly results from GCP

In [ ]:
os.chdir(result_dir)
!gsutil -m -o 'GSUtil:parallel_composite_upload_threshold=150M' -o 'GSUtil:parallel_process_count=4' -o 'GSUtil:parallel_thread_count=4' rsync -x '.*\.fastq\.gz' gs://{out_bucket}/ ./
!gzip Trinity.fasta